# Imports

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
from nltk.corpus import stopwords

# Reading reviews

In [ ]:
reviews_train = []
for line in open('C:/Users/kurti/Desktop/aclImdb/movie_data/full_train.txt', encoding='utf-8'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('C:/Users/kurti/Desktop/aclImdb/movie_data/full_test.txt', encoding='utf-8'):
    
    reviews_test.append(line.strip())
    
target = [1 if i < 12500 else 0 for i in range(25000)]

# Cleaning Regex

In [ ]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

# Remove Stop Words 

In [ ]:
english_stop_words = set(stopwords.words('english'))

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)
stop_words = ['in', 'of', 'at', 'a', 'the']


# N gram

In [ ]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

# SVM

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    # lr = LogisticRegression(C=c)
    svm = LinearSVC(C=c, max_iter=5000)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))

# Final Accuracy

In [ ]:
final_ngram = LinearSVC(C=0.01)
final_ngram.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_ngram.predict(X_test)))

# Final Accuracy with Baseline, no stop words , no ngram, c = 0.05: 0.88144
# Final Accuracy with Stopwords , c = 0.5 = 0.87972
# Final Accuracy with Stop words + Ngram, c = 0.5 = 0.89868
# Final Accuracy with Stop words + Ngram + SVM, c= 0.5 = 0.89412
# Final Accuracy with Stop words + Ngram + SVM, c = 0.01 = 0.89769
# Final Accuracy with custom Stop words + Ngram + SVM, c = 0.01 = 0.9 (BEST VALUE FOR C)

# Debug Statements 

In [ ]:
reviews_train[10]

In [ ]:
reviews_train_clean[10]